# Metadata

**L1 Taxonomy** - Software Architecture & Design

**L2 Taxonomy** - Factory Pattern

**Subtopic** - Integrating factories with dependency injection: injecting a fake factory for testing to produce dummy objects

**Use Case** - Develop a simple calculator application in Python. The calculator will support basic operations like addition, subtraction, multiplication, and division. Implement the operations using the Factory Pattern where each operation is a separate class. Then, create a factory class that, based on the operation input, returns the corresponding operation class. Integrate the factory class with dependency injection, so that a fake factory can be injected for testing purposes. The fake factory will produce dummy operation classes that mimic the behavior of the real operation classes but return predefined results.

**Programming Language** - Python

**Target Model** - o1

# Setup

```requirements.txt
```


# Prompt

## Problem Description

You're building a payment processing system for an e-commerce platform that needs to handle multiple payment methods. The system must use the Factory Pattern with Dependency Injection to create payment processors dynamically and support both production and testing environments with configurable mock processors.

## Input
- `payment_method`: string (payment type identifier)
- `amount`: float (transaction amount)
- `currency`: string (3-letter currency code)
- `customer_info`: dict (payment method specific data)

## Output
Dictionary with transaction result:
```python
{
    "success": bool,
    "transaction_id": str,
    "message": str,
    "fee": float
}
```

## Examples

**Example 1:**
```python
Input:
payment_method = "credit_card"
amount = 100.00
currency = "USD"
customer_info = {"customer_id": "CUST001", "card_number": "****1234"}

Output:
{
    "success": True,
    "transaction_id": "CC_20250730235257_CUST001",
    "message": "Credit card payment of 100.0 USD processed successfully",
    "fee": 3.20
}
# Explanation: Credit card processed with 2.9% + $0.30 fee = $3.20
```

**Example 2:**
```python
Input:
payment_method = "paypal"
amount = 50.00
currency = "JPY"
customer_info = {"customer_id": "CUST002", "email": "user@example.com"}

Output:
{
    "success": False,
    "transaction_id": "",
    "message": "Invalid payment parameters",
    "fee": 0.0
}
```

## Function Signature
```python
def process_transaction(payment_method: str, amount: float,
                       currency: str, customer_info: Dict) -> Dict:
    pass

class PaymentProcessor(ABC):
    def process_payment(self, amount: float, currency: str,
                       customer_info: Dict) -> PaymentResult:
        pass

class PaymentProcessorFactory(ABC):
    def create_processor(self, payment_method: str) -> PaymentProcessor:
        pass

class PaymentGateway:
    def __init__(self, processor_factory: PaymentProcessorFactory):
        pass
```


# Requirements

## Explicit Requirements
- Implement abstract base classes for PaymentProcessor and PaymentProcessorFactory
- Create concrete processors for credit_card, paypal, and bank_transfer
- Use dependency injection to pass factory to PaymentGateway
- Implement both StandardPaymentProcessorFactory and FakePaymentProcessorFactory
- Return standardized response format for all payment methods
- Generate unique transaction IDs with format: {METHOD}_{TIMESTAMP}_{CUSTOMER_ID}
- Log all transactions with timestamp

## Implicit Requirements
- Follow Factory Pattern correctly
- Ensure testability through dependency injection
- FakePaymentProcessorFactory must allow setting custom results and fees
- Each processor validates its specific required customer_info fields
- Maintain separation between production and test implementations

## Constraints
- Amount must be positive (> 0)
- Currency must be exactly 3 uppercase letters
- Credit card: supports USD, EUR, GBP; fee = 2.9% + $0.30
- PayPal: supports USD, EUR, GBP, CAD, AUD; fee = 3.49% + $0.49
- Bank transfer: supports USD, EUR; minimum $100,  flat fee = $15
- Invalid payment method raises ValueError
- Transaction IDs must be unique

## Edge Cases
- Zero or negative amounts
- Unsupported currency for payment method
- Missing required customer_info fields (card_number for credit_card, email for paypal, account_number and routing_number for bank_transfer)
- Invalid payment method name
- Amount below minimum for bank transfers
- Empty or None customer_info
- FakeFactory simulating failures

In [ ]:
# code

"""Payment gateway factory implementation."""

from abc import ABC, abstractmethod
from typing import Dict
from datetime import datetime
from dataclasses import dataclass


@dataclass
class PaymentResult:
    """Data class for payment results."""

    success: bool
    transaction_id: str
    message: str
    fee: float = 0.0

    def to_dict(self) -> Dict:
        """Convert PaymentResult to dictionary."""
        return {
            "success": self.success,
            "transaction_id": self.transaction_id,
            "message": self.message,
            "fee": self.fee,
        }


class PaymentProcessor(ABC):
    """Abstract base class for payment processors."""

    @abstractmethod
    def process_payment(
        self, amount: float, currency: str, customer_info: Dict
    ) -> PaymentResult:
        """Process a payment transaction."""
        pass


class PaymentProcessorFactory(ABC):
    """Abstract factory for creating payment processors."""

    @abstractmethod
    def create_processor(self, payment_method: str) -> PaymentProcessor:
        """Create a payment processor for the given method."""
        pass


class CreditCardProcessor(PaymentProcessor):
    """Credit card payment processor."""

    def __init__(self):
        """Initialize credit card processor."""
        self.supported_currencies = {"USD", "EUR", "GBP"}
        self.fee_percentage = 0.029
        self.flat_fee = 0.30

    def process_payment(
        self, amount: float, currency: str, customer_info: Dict
    ) -> PaymentResult:
        """Process credit card payment."""
        if amount <= 0:
            return PaymentResult(False, "", "Invalid payment parameters", 0.0)

        if (
            not self._is_valid_currency(currency)
            or currency not in self.supported_currencies
        ):
            return PaymentResult(False, "", "Invalid payment parameters", 0.0)

        if not customer_info or "card_number" not in customer_info:
            return PaymentResult(False, "", "Missing card information", 0.0)

        fee = round(amount * self.fee_percentage + self.flat_fee, 2)
        customer_id = customer_info.get("customer_id", "unknown")
        transaction_id = (
            f"CC_{datetime.now().strftime('%Y%m%d%H%M%S')}_{customer_id}"
        )

        return PaymentResult(
            True,
            transaction_id,
            f"Credit card payment of {amount} {currency} processed "
            "successfully",
            fee,
        )

    def _is_valid_currency(self, currency: str) -> bool:
        """Validate currency format."""
        return (
            isinstance(currency, str)
            and len(currency) == 3
            and currency.isupper()
        )


class PayPalProcessor(PaymentProcessor):
    """PayPal payment processor."""

    def __init__(self):
        """Initialize PayPal processor."""
        self.supported_currencies = {"USD", "EUR", "GBP", "CAD", "AUD"}
        self.fee_percentage = 0.0349
        self.flat_fee = 0.49

    def process_payment(
        self, amount: float, currency: str, customer_info: Dict
    ) -> PaymentResult:
        """Process PayPal payment."""
        if amount <= 0:
            return PaymentResult(False, "", "Invalid payment parameters", 0.0)

        if (
            not self._is_valid_currency(currency)
            or currency not in self.supported_currencies
        ):
            return PaymentResult(False, "", "Invalid payment parameters", 0.0)

        if not customer_info or "email" not in customer_info:
            return PaymentResult(False, "", "Missing PayPal email", 0.0)

        fee = round(amount * self.fee_percentage + self.flat_fee, 2)
        customer_id = customer_info.get("customer_id", "unknown")
        transaction_id = (
            f"PP_{datetime.now().strftime('%Y%m%d%H%M%S')}_{customer_id}"
        )

        return PaymentResult(
            True,
            transaction_id,
            f"PayPal payment of {amount} {currency} processed successfully",
            fee,
        )

    def _is_valid_currency(self, currency: str) -> bool:
        """Validate currency format."""
        return (
            isinstance(currency, str)
            and len(currency) == 3
            and currency.isupper()
        )


class BankTransferProcessor(PaymentProcessor):
    """Bank transfer payment processor."""

    def __init__(self):
        """Initialize bank transfer processor."""
        self.supported_currencies = {"USD", "EUR"}
        self.flat_fee = 15.0
        self.minimum_amount = 100.0

    def process_payment(
        self, amount: float, currency: str, customer_info: Dict
    ) -> PaymentResult:
        """Process bank transfer payment."""
        if amount <= 0 or amount < self.minimum_amount:
            return PaymentResult(False, "", "Invalid payment parameters", 0.0)

        if (
            not self._is_valid_currency(currency)
            or currency not in self.supported_currencies
        ):
            return PaymentResult(False, "", "Invalid payment parameters", 0.0)

        if (
            not customer_info
            or "account_number" not in customer_info
            or "routing_number" not in customer_info
        ):
            return PaymentResult(
                False, "", "Missing bank account information", 0.0
            )

        customer_id = customer_info.get("customer_id", "unknown")
        transaction_id = (
            f"BT_{datetime.now().strftime('%Y%m%d%H%M%S')}_{customer_id}"
        )

        return PaymentResult(
            True,
            transaction_id,
            f"Bank transfer of {amount} {currency} initiated successfully",
            self.flat_fee,
        )

    def _is_valid_currency(self, currency: str) -> bool:
        """Validate currency format."""
        return (
            isinstance(currency, str)
            and len(currency) == 3
            and currency.isupper()
        )


class StandardPaymentProcessorFactory(PaymentProcessorFactory):
    """Production factory implementation."""

    def __init__(self):
        """Initialize standard payment processor factory."""
        self._processors = {
            "credit_card": CreditCardProcessor,
            "paypal": PayPalProcessor,
            "bank_transfer": BankTransferProcessor,
        }

    def create_processor(self, payment_method: str) -> PaymentProcessor:
        """Create processor for the given payment method."""
        processor_class = self._processors.get(payment_method)
        if not processor_class:
            raise ValueError(f"Unsupported payment method: {payment_method}")
        return processor_class()


class FakePaymentProcessorFactory(PaymentProcessorFactory):
    """Test factory implementation with configurable results."""

    def __init__(self):
        """Initialize fake payment processor factory."""
        self._mock_results = {}
        self._mock_fees = {}

    def set_mock_result(
        self,
        payment_method: str,
        success: bool,
        transaction_id: str = "",
        message: str = "",
    ):
        """Configure mock result for a payment method."""
        self._mock_results[payment_method] = {
            "success": success,
            "transaction_id": transaction_id,
            "message": message,
        }

    def set_mock_fee(self, payment_method: str, fee: float):
        """Configure mock fee for a payment method."""
        self._mock_fees[payment_method] = fee

    def create_processor(self, payment_method: str) -> PaymentProcessor:
        """Create mock processor for the given payment method."""
        if payment_method not in self._mock_results:
            raise ValueError(f"Unsupported payment method: {payment_method}")

        class MockProcessor(PaymentProcessor):
            def __init__(self, result_config, fee):
                self.result_config = result_config
                self.fee = fee

            def process_payment(
                self, amount: float, currency: str, customer_info: Dict
            ) -> PaymentResult:
                return PaymentResult(
                    self.result_config["success"],
                    self.result_config["transaction_id"],
                    self.result_config["message"],
                    self.fee,
                )

        return MockProcessor(
            self._mock_results[payment_method],
            self._mock_fees.get(payment_method, 0.0),
        )


class PaymentGateway:
    """Main payment gateway with dependency injection."""

    def __init__(self, processor_factory: PaymentProcessorFactory):
        """Initialize payment gateway."""
        self._factory = processor_factory
        self._transaction_log = []

    def process_transaction(
        self,
        payment_method: str,
        amount: float,
        currency: str,
        customer_info: Dict,
    ) -> Dict:
        """Process a payment transaction."""
        try:
            processor = self._factory.create_processor(payment_method)
            result = processor.process_payment(amount, currency, customer_info)

            self._transaction_log.append(
                {
                    "timestamp": datetime.now().isoformat(),
                    "payment_method": payment_method,
                    "amount": amount,
                    "currency": currency,
                    "result": result.to_dict(),
                }
            )

            return result.to_dict()

        except ValueError as e:
            return {
                "success": False,
                "transaction_id": "",
                "message": str(e),
                "fee": 0.0,
            }


def process_transaction(
    payment_method: str, amount: float, currency: str, customer_info: Dict
) -> Dict:
    """Process a transaction using the standard factory."""
    factory = StandardPaymentProcessorFactory()
    gateway = PaymentGateway(factory)
    return gateway.process_transaction(
        payment_method, amount, currency, customer_info
    )


In [ ]:
# tests


# Model Breaking Proof

#### Model Breaking Task URL: <https://labeling-ms.turing.com/conversations/93547/view>

#### Model code:

```python
# code generated by the model

import datetime
from abc import ABC, abstractmethod
from typing import Dict

###############################################################################
# Data Structure for Payment Result
###############################################################################
def make_payment_result(success: bool, transaction_id: str,
                        message: str, fee: float) -> Dict:
    """
    Helper function to standardize the transaction result output.
    """
    return {
        "success": success,
        "transaction_id": transaction_id,
        "message": message,
        "fee": fee
    }

# <Issue>: Missing PaymentResult class - function signature requires PaymentResult return type, not Dict

###############################################################################
# Abstract Base Classes
###############################################################################
class PaymentProcessor(ABC):
    """
    Abstract PaymentProcessor specifying the interface for processing payments.
    """
    @abstractmethod
    def process_payment(self, amount: float, currency: str,
                        customer_info: Dict) -> Dict:  # <Issue>: Should return PaymentResult, not Dict
        """
        Processes the payment and returns a dictionary structure:
        {
            "success": bool,
            "transaction_id": str,
            "message": str,
            "fee": float
        }
        """
        pass


class PaymentProcessorFactory(ABC):
    """
    Abstract Factory specifying how payment processors are created.
    """
    @abstractmethod
    def create_processor(self, payment_method: str) -> PaymentProcessor:
        """
        Returns the appropriate PaymentProcessor for the given method.
        """
        pass

###############################################################################
# Concrete Payment Processors
###############################################################################
class CreditCardProcessor(PaymentProcessor):
    """
    Processor for credit card transactions.
    Supported currencies: USD, EUR, GBP
    Fee structure: 2.9% + $0.30
    """
    SUPPORTED_CURRENCIES = {"USD", "EUR", "GBP"}
    PERCENT_FEE = 0.029
    FIXED_FEE = 0.30

    def process_payment(self, amount: float, currency: str,
                        customer_info: Dict) -> Dict:
        # Basic validations
        if amount <= 0:
            return make_payment_result(False, "", "Amount must be positive", 0.0)  # <Issue>: Should return "Invalid payment parameters"
        if currency not in self.SUPPORTED_CURRENCIES:
            return make_payment_result(False, "", f"Unsupported currency {currency}", 0.0)  # <Issue>: Should return "Invalid payment parameters"
        if not customer_info or "card_number" not in customer_info:
            return make_payment_result(False, "", "Missing required card_number", 0.0)  # <Issue>: Should return "Missing card information"
        
        # <Issue>: Missing validation that currency is 3 uppercase letters
        
        # Compute fee
        fee = amount * self.PERCENT_FEE + self.FIXED_FEE
        
        # Generate transaction_id
        customer_id = customer_info.get("customer_id", "UNKNOWN")  # <Issue>: Should use "unknown" not "UNKNOWN"
        transaction_id = self._generate_transaction_id("CC", customer_id)

        # Construct success message
        message = f"Credit card payment of {amount} {currency} processed successfully"

        # Return result
        return make_payment_result(True, transaction_id, message, round(fee, 2))

    def _generate_transaction_id(self, method: str, customer_id: str) -> str:
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        return f"{method}_{timestamp}_{customer_id}"


class PayPalProcessor(PaymentProcessor):
    """
    Processor for PayPal transactions.
    Supported currencies: USD, EUR, GBP, CAD, AUD
    Fee structure: 3.49% + $0.49
    """
    SUPPORTED_CURRENCIES = {"USD", "EUR", "GBP", "CAD", "AUD"}
    PERCENT_FEE = 0.0349
    FIXED_FEE = 0.49

    def process_payment(self, amount: float, currency: str,
                        customer_info: Dict) -> Dict:
        # Basic validations
        if amount <= 0:
            return make_payment_result(False, "", "Amount must be positive", 0.0)  # <Issue>: Should return "Invalid payment parameters"
        if currency not in self.SUPPORTED_CURRENCIES:
            return make_payment_result(False, "", f"Unsupported currency {currency}", 0.0)  # <Issue>: Should return "Invalid payment parameters"
        if not customer_info or "email" not in customer_info:
            return make_payment_result(False, "", "Missing required email", 0.0)  # <Issue>: Should return "Missing PayPal email"

        # <Issue>: Missing validation that currency is 3 uppercase letters

        # Compute fee
        fee = amount * self.PERCENT_FEE + self.FIXED_FEE
        
        # Generate transaction_id
        customer_id = customer_info.get("customer_id", "UNKNOWN")  # <Issue>: Should use "unknown" not "UNKNOWN"
        transaction_id = self._generate_transaction_id("PP", customer_id)

        # Construct success message
        message = f"PayPal payment of {amount} {currency} processed successfully"

        # Return result
        return make_payment_result(True, transaction_id, message, round(fee, 2))

    def _generate_transaction_id(self, method: str, customer_id: str) -> str:
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        return f"{method}_{timestamp}_{customer_id}"


class BankTransferProcessor(PaymentProcessor):
    """
    Processor for bank transfer transactions.
    Supported currencies: USD, EUR
    Minimum amount: 100
    Flat fee: $15
    """
    SUPPORTED_CURRENCIES = {"USD", "EUR"}
    FLAT_FEE = 15.0
    MINIMUM_AMOUNT = 100.0

    def process_payment(self, amount: float, currency: str,
                        customer_info: Dict) -> Dict:
        # Basic validations
        if amount < self.MINIMUM_AMOUNT:  # <Issue>: Also needs to check amount <= 0
            return make_payment_result(False, "",
                f"Bank transfer requires a minimum of {self.MINIMUM_AMOUNT}", 0.0)  # <Issue>: Should return "Invalid payment parameters"
        if currency not in self.SUPPORTED_CURRENCIES:
            return make_payment_result(False, "", f"Unsupported currency {currency}", 0.0)  # <Issue>: Should return "Invalid payment parameters"
        
        # <Issue>: Missing validation that currency is 3 uppercase letters
        
        required_fields = {"account_number", "routing_number"}
        if not customer_info or not required_fields.issubset(customer_info.keys()):
            return make_payment_result(False, "",
                    "Missing required bank transfer details", 0.0)  # <Issue>: Should return "Missing bank account information"

        # Fee is flat
        fee = self.FLAT_FEE
        
        # Generate transaction_id
        customer_id = customer_info.get("customer_id", "UNKNOWN")  # <Issue>: Should use "unknown" not "UNKNOWN"
        transaction_id = self._generate_transaction_id("BT", customer_id)

        # Construct success message
        message = f"Bank transfer of {amount} {currency} processed successfully"  # <Issue>: Should say "initiated successfully"

        # Return result
        return make_payment_result(True, transaction_id, message, fee)

    def _generate_transaction_id(self, method: str, customer_id: str) -> str:
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        return f"{method}_{timestamp}_{customer_id}"

###############################################################################
# Concrete Payment Processor Factories
###############################################################################
class StandardPaymentProcessorFactory(PaymentProcessorFactory):
    """
    Factory that creates the standard (production) payment processors.
    """
    def create_processor(self, payment_method: str) -> PaymentProcessor:
        payment_method = payment_method.lower()
        if payment_method == "credit_card":
            return CreditCardProcessor()
        elif payment_method == "paypal":
            return PayPalProcessor()
        elif payment_method == "bank_transfer":
            return BankTransferProcessor()
        else:
            raise ValueError(f"Invalid payment method: {payment_method}")  # <Issue>: Error message should be "Unsupported payment method: {payment_method}"

class FakePaymentProcessorFactory(PaymentProcessorFactory):
    """
    Factory that creates fake (mock) payment processors, useful for testing.
    It allows setting custom results or simulating failures.
    """
    def __init__(self, default_success=True, default_fee=0.0,
                 failure_message="Simulated failure"):
        """
        :param default_success: whether the mock processors succeed by default
        :param default_fee: default fee to simulate
        :param failure_message: message if simulating failure
        """
        self.default_success = default_success
        self.default_fee = default_fee
        self.failure_message = failure_message
        
    # <Issue>: Missing set_mock_result() and set_mock_fee() methods required by specifications

    def create_processor(self, payment_method: str) -> PaymentProcessor:
        return FakeProcessor(
            success=self.default_success,
            fee=self.default_fee,
            failure_message=self.failure_message
        )

class FakeProcessor(PaymentProcessor):
    """
    A fake processor that returns a configurable result
    (success/failure) for testing purposes.
    """
    def __init__(self, success: bool, fee: float, failure_message: str):
        self.success = success
        self.fee = fee
        self.failure_message = failure_message

    def process_payment(self, amount: float, currency: str,
                        customer_info: Dict) -> Dict:
        if self.success:
            transaction_id = f"FAKE_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"
            return make_payment_result(
                True,
                transaction_id,
                f"Fake payment of {amount} {currency} succeeded (mock).",
                self.fee
            )
        else:
            return make_payment_result(
                False,
                "",
                self.failure_message,
                0.0
            )

###############################################################################
# Payment Gateway
###############################################################################
class PaymentGateway:
    """
    Uses a PaymentProcessorFactory to create processors and handle payments.
    Demonstrates Dependency Injection (the factory is injected).
    """
    def __init__(self, processor_factory: PaymentProcessorFactory):
        self.processor_factory = processor_factory
        # <Issue>: Missing transaction log storage - requirement states "Maintain a transaction log for all processed payments"

    def process_payment(self, payment_method: str, amount: float,  # <Issue>: Method should be named process_transaction
                        currency: str, customer_info: Dict) -> Dict:
        # Create the processor from the factory
        processor = self.processor_factory.create_processor(payment_method)
        # Process the payment
        result = processor.process_payment(amount, currency, customer_info)
        # Log the transaction (here we simply print to console)
        self._log_transaction(payment_method, result)  # <Issue>: Should store logs, not just print
        return result

    def _log_transaction(self, payment_method: str, result: Dict):
        """
        Logs the transaction details with a timestamp.
        """
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"[{timestamp}] Method: {payment_method}, "
              f"Success: {result['success']}, "
              f"TxnID: {result['transaction_id']}, "
              f"Message: {result['message']}, "
              f"Fee: {result['fee']}")

###############################################################################
# High-Level Function
###############################################################################
def process_transaction(payment_method: str, amount: float,
                        currency: str, customer_info: Dict) -> Dict:
    """
    High-level function to process a transaction using the standard
    (production) payment factory.
    This function returns a dictionary of the transaction result:
    {
        "success": bool,
        "transaction_id": str,
        "message": str,
        "fee": float
    }
    """
    # Simple currency format check
    if len(currency) != 3 or not currency.isalpha():
        return make_payment_result(False, "",
                                   "Currency code must be 3 letters.", 0.0)  # <Issue>: Should check for uppercase and return "Invalid payment parameters"

    # We inject the standard factory into our PaymentGateway.
    gateway = PaymentGateway(StandardPaymentProcessorFactory())
    return gateway.process_payment(payment_method, amount, currency.upper(), customer_info)  # <Issue>: Should call gateway.process_transaction

###############################################################################
# Example Usage
###############################################################################
if __name__ == "__main__":
    # Example 1
    result1 = process_transaction(
        payment_method="credit_card",
        amount=100.00,
        currency="USD",
        customer_info={"customer_id": "CUST001", "card_number": "****1234"}
    )
    print("Result 1:", result1)

    # Example 2
    result2 = process_transaction(
        payment_method="paypal",
        amount=50.00,
        currency="JPY",
        customer_info={"customer_id": "CUST002", "email": "user@example.com"}
    )
    print("Result 2:", result2)
```